In [20]:
import requests
import pandas as pd

In [21]:
# # def __get_distance_osrm(self, source_lat, source_long, destination_lat, destination_long):
# destination_lat=25.404874	
# destination_long=87.867901
# source_lat=22.564566
# source_long=88.39582

# try:
#     source_cord = str(source_long) + ',' + str(source_lat)
#     destination_cord = str(destination_long) + ',' + str(destination_lat)
#     url = 'http://router.project-osrm.org/route/v1/driving/{0};{1}?steps=true'.format(source_cord,destination_cord)
#     # url = 'http://13.234.43.137:5000/route/v1/driving/' + source_cord + ';' + destination_cord
#     req = requests.get(url)
#     response = req.json()
#     print(response['routes'][0]['legs'][0]['duration']/3600)
# except Exception as e:
#     print("Error:", e)

In [22]:
data=pd.read_excel("data_1.xlsx",engine='openpyxl')
# !pip install openpyxl

In [23]:
node_mother=data.query('Category=="Mother"').reset_index(drop=True)
node_daughter=data.query('Category=="Daughter"').reset_index(drop=True)

In [24]:

node_daughter.head()

,center_name,latitude,longitude,Category
0,1,25.404874,87.867901,Daughter
1,2,23.250398,88.333175,Daughter
2,3,25.676970,88.218262,Daughter
3,4,22.782778,88.576250,Daughter
4,5,23.531450,86.668270,Daughter


In [29]:
len(node_daughter)

191

In [46]:
node_list = [*node_mother.center_name.values]
DC = [*node_daughter.center_name.values]
total_nodes=node_list.copy()
total_nodes.extend(DC)
# node_list.extend(DC)
# # dc_list = node_list
dc_list = total_nodes
dist_matrix = {i:{j:0 for j in dc_list} for i in dc_list}
time_matrix = {i:{j:0 for j in dc_list} for i in dc_list}

In [56]:
node_list

[57, 118, 124, 133, 134, 137, 141, 142, 145, 171, 178]

In [ ]:
n=0
for i in dc_list:
    origin = (data.loc[data.center_name==i,'latitude'].values[0],data.loc[data.center_name==i,'longitude'].values[0])
    print(n)
    n+=1
    for j in dc_list:
        try:
            destination =(data.loc[data.center_name==j,'latitude'].values[0],data.loc[data.center_name==j,'longitude'].values[0])
            # if self.USE_GOOGLE_API is True:
            #     distance = (self.gmaps.distance_matrix(origin, destination, mode='driving', avoid='ferries')['rows'][0]['elements'][0]['distance']['value'])/1000
            #     time = (self.gmaps.distance_matrix(origin, destination, mode='driving', avoid='ferries')['rows'][0]['elements'][0]['duration']['value'])/3600
            # if distance>self.model_parameters['MAX_DISTANCE']:
            #     print(f'Warning!!! Distance between {i} and {j} seems high')
            # time_matrix[i][j] = time
            # OSRM
            source_cord = str(origin[1])+','+str(origin[0])
            destination_cord = str(destination[1])+','+str(destination[0])
            # url = 'http://router.project-osrm.org/route/v1/driving/'+source_cord+';'+destination_cord+'?steps=true'
            url=  'http://router.project-osrm.org/route/v1/driving/{0};{1}?steps=true'.format(source_cord,destination_cord)
            req = requests.get(url)
            response = req.json()
            distance = response['routes'][0]['legs'][0]['distance']/1000
                    #    response['routes'][0]['legs'][0]['duration']
            # print(distance)
            dist_matrix[i][j] = distance
        
        except:
            print(f'Could not compute distance for {j}')
print('Distance computation done...')

In [ ]:
results_distance = pd.DataFrame(dist_matrix, index = dist_matrix.keys(), columns = dist_matrix.keys())
results_distance

In [63]:
# results_distance.to_csv('dist_matrix_df.csv',index=False)
import pandas as pd
results_time=pd.read_csv('time_matrix_df.csv',index_col=None)
# results_distance

In [66]:
time_matrix_list = []
for i,k in results_time.iterrows():
    time_matrix_list.append(list(results_time.loc[i,:].values))

In [65]:

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [67]:
def create_data_model():
    """Stores the data for xthe problem."""
    data = {}
    # data['distance_matrix'] =dist_matrix_list
    # data['distance_matrix'] =dist_matrix_list
    data['time_matrix']=time_matrix_list
    data['num_vehicles'] = 11
    data['depot'] = 0
    data['starts']=[57, 118, 124, 133, 134, 137, 141, 142, 145, 171, 178]
    data['ends']=[57, 118, 124, 133, 134, 137, 141, 142, 145, 171, 178]
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    # manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
    #                                        data['num_vehicles'], data['depot'])
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['starts'],data['ends'])
    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # # Create and register a transit callback.
    # def distance_callback(from_index, to_index):
    #     """Returns the distance between the two nodes."""
    #     # Convert from routing variable Index to distance matrix NodeIndex.
    #     from_node = manager.IndexToNode(from_index)
    #     to_node = manager.IndexToNode(to_index)
    #     return data['distance_matrix'][from_node][to_node]
    # transit_callback_index = routing.RegisterTransitCallback(distance_callback)


    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # # Add Distance constraint.
    # dimension_name = 'Distance'
    # routing.AddDimension(
    #     transit_callback_index,
    #     0,  # no slack
    #     3000,  # vehicle maximum travel distance
    #     True,  # start cumul to zero
    #     dimension_name)
    # distance_dimension = routing.GetDimensionOrDie(dimension_name)
    # distance_dimension.SetGlobalSpanCostCoefficient(100)
# Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        30,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    search_parameters.time_limit.seconds=30
    search_parameters.solution_limit=100

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No solution found !')


if __name__ == '__main__':
    main()

Objective: 0
Route for vehicle 0:
 57 -> 57
Distance of the route: 0m

Route for vehicle 1:
 118 -> 118
Distance of the route: 0m

Route for vehicle 2:
 124 -> 124
Distance of the route: 0m

Route for vehicle 3:
 133 -> 133
Distance of the route: 0m

Route for vehicle 4:
 134 -> 134
Distance of the route: 0m

Route for vehicle 5:
 137 -> 137
Distance of the route: 0m

Route for vehicle 6:
 141 -> 141
Distance of the route: 0m

Route for vehicle 7:
 142 -> 142
Distance of the route: 0m

Route for vehicle 8:
 145 -> 145
Distance of the route: 0m

Route for vehicle 9:
 171 -> 171
Distance of the route: 0m

Route for vehicle 10:
 178 ->  201 ->  200 ->  199 ->  198 ->  197 ->  196 ->  195 ->  194 ->  193 ->  192 ->  191 ->  190 ->  189 ->  188 ->  187 ->  186 ->  185 ->  184 ->  183 ->  182 ->  181 ->  180 ->  179 ->  177 ->  176 ->  175 ->  174 ->  173 ->  172 ->  170 ->  169 ->  168 ->  167 ->  166 ->  165 ->  164 ->  163 ->  162 ->  161 ->  160 ->  159 ->  158 ->  157 ->  156 ->  155 ->